In [0]:
# TensorFlow and keras
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.models import Sequential
# Helper libraries
import os
from glob import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
import cv2
from keras.models import Model
from keras.layers import add
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras.utils import plot_model
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate
)
import imgaug as ia
from imgaug import augmenters as iaa

In [0]:
!pip install import-ipynb

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd "/content/drive/My Drive/Colab Notebooks"
import import_ipynb

In [0]:
import data_loader as data
import Models as models

In [0]:
training_generator = data.DataGenerator('/content/drive/My Drive/MRNet-v1.0/', 'meniscus', 'coronal', augment=True, train=True)
validation_generator = data.DataGenerator('/content/drive/My Drive/MRNet-v1.0/', 'meniscus', 'coronal', augment=False, train=False)
#model.fit_generator(generator=training_generator, validation_data=validation_generator,use_multiprocessing=True,workers=11)

In [0]:
modelVGG = models.build_model_VGG()
modelVGG.compile(optimizer='adam',
               loss = tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
modelVGG.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 3)       84        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 3)       84        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 3)       0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 18, 18, 3)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 972)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1946  

In [0]:
EPOCHS = 10
early_stop = tf.keras.callbacks.EarlyStopping(patience=2)
historyVGG = modelVGG.fit_generator(
    training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks = [early_stop]
)

Epoch 1/10
1130/1130 [==============================] - 60s 53ms/step - loss: 0.4973 - accuracy: 0.8351 - val_loss: 0.1569 - val_accuracy: 0.8079
Epoch 2/10
1130/1130 [==============================] - 48s 43ms/step - loss: 0.4477 - accuracy: 0.8356 - val_loss: 0.2021 - val_accuracy: 0.8167
Epoch 3/10
1130/1130 [==============================] - 48s 43ms/step - loss: 0.4267 - accuracy: 0.8323 - val_loss: 0.2210 - val_accuracy: 0.8164


In [0]:
modelResNet = models.build_model_ResNet()
modelResNet.compile(optimizer='adam',
               loss = tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
modelResNet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 3)  84          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 3)  84          conv2d_1[0][0]                   
__________________________________________________________________________________________________
add_1 (Add)                     (None, 256, 256, 3)  0           conv2d_2[0][0]                   
                                                                 input_1[0][0]                

In [0]:
EPOCHS = 10
early_stop = tf.keras.callbacks.EarlyStopping(patience=2)
historyResNet = modelResNet.fit_generator(
    training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks = [early_stop]
)

Epoch 1/10
1130/1130 [==============================] - 58s 51ms/step - loss: 0.4828 - accuracy: 0.8311 - val_loss: 0.3096 - val_accuracy: 0.8086
Epoch 2/10
1130/1130 [==============================] - 47s 42ms/step - loss: 0.4525 - accuracy: 0.8293 - val_loss: 0.1728 - val_accuracy: 0.8099
Epoch 3/10
1130/1130 [==============================] - 47s 42ms/step - loss: 0.4286 - accuracy: 0.8342 - val_loss: 0.3414 - val_accuracy: 0.8048
Epoch 4/10
1130/1130 [==============================] - 48s 42ms/step - loss: 0.4076 - accuracy: 0.8394 - val_loss: 0.2645 - val_accuracy: 0.8062


In [0]:
modelV3 = models.build_model_inception()
modelV3.compile(optimizer='adam',
               loss = tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
modelV3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 256, 256, 64) 256         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 256, 256, 128 3584        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 256, 256, 32) 2432        input_2[0][0]                    
______________________________________________________________________________________________

In [0]:
EPOCHS = 10
early_stop = tf.keras.callbacks.EarlyStopping(patience=2)
historyV3 = modelV3.fit_generator(
    training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks = [early_stop]
)

Epoch 1/10
1130/1130 [==============================] - 744s 658ms/step - loss: 0.8407 - accuracy: 0.7672 - val_loss: 0.6440 - val_accuracy: 0.6574
Epoch 2/10
1130/1130 [==============================] - 245s 217ms/step - loss: 0.6658 - accuracy: 0.7802 - val_loss: 1.4165 - val_accuracy: 0.3737
Epoch 3/10
1130/1130 [==============================] - 248s 219ms/step - loss: 0.6634 - accuracy: 0.7890 - val_loss: 0.2307 - val_accuracy: 0.8120
Epoch 4/10
1130/1130 [==============================] - 248s 219ms/step - loss: 0.6545 - accuracy: 0.7906 - val_loss: 0.6281 - val_accuracy: 0.7564
Epoch 5/10
1130/1130 [==============================] - 249s 220ms/step - loss: 0.6277 - accuracy: 0.8054 - val_loss: 0.0316 - val_accuracy: 0.8179
Epoch 6/10
1130/1130 [==============================] - 248s 220ms/step - loss: 0.6214 - accuracy: 0.8104 - val_loss: 0.0147 - val_accuracy: 0.8106
Epoch 7/10
1130/1130 [==============================] - 248s 219ms/step - loss: 0.6140 - accuracy: 0.8159 - val_